<center><h1><b><span style="color:blue">Particles, decays, HEP units</span></b></h1></center>

&nbsp;<br>
### **Quick intro to the following packages**
- `hepunits` - the HEP system of units.
- `Particle` - PDG particle data, MC identification codes, and more.
- `DecayLanguage` - Decay files (notably for EvtGen), universal description of decay chains.

<center><h2><b><span style="color:green">hepunits - The HEP system of units</span></b></h2></center>

The package ``hepunits`` collects the most commonly used units and constants in the
HEP System of Units, which are *not* the same as the international system of units (aka SI units).
The HEP system of units is based on the following:

| Quantity            | Name               | Unit|
| ------------------ :| ----------------- :| -- :|
|  Length             |  millimeter        | mm  |
|  Time               |  nanosecond        | ns  |
|  Energy             |  Mega electron Volt| MeV |
|  Positron charge    |  eplus             |     |
|  Temperature        |  kelvin            | K   |
|  Amount of substance|  mole              | mol |
|  Luminous intensity |  candela           | cd  |
| Plane angle         |  radian            | rad |
|  Solid angle        |  steradian         | sr  |

Note: no need to make use of sophisticated packages (e.g. as in AstroPy) since we basically never need to change systems of units (we never use ergs as energy, for example ;-)).

**Basic usage is straightforward, though it may be confusing at first. Remember, all variables are written wrt to the units:**

In [ ]:
from hepunits import mm, ns, MeV, eplus, GeV, kelvin, mol, cd, rad, sr

mm == ns == MeV == eplus == kelvin == mol == cd == rad == sr == 1

In [ ]:
GeV == 1000*MeV

Add two quantities with different length units:

In [ ]:
from hepunits import units as u

1*u.meter + 5*u.cm

Indeed, the result is in HEP units, so mm. Rather obtain the result in meters:

In [ ]:
(1*u.meter + 5*u.cm) / u.meter

Do you need to play a bit more to get a proper feeling? This next (non-academic) exercise should help you ...

### **Quick time-of-flight study**

Let's try to play with units in a meaningful way, in a kind of exercise that physicists encounter. Imagine you are investigating time-of-flight (ToF) detectors for particle identification. The time it takes a particle of velocity $\beta = v/c= pc/E$ to travel a distance $L$ is given by

$$\mathrm{ToF} = \frac{L}{c \beta}$$

It results that the mass $m$ of the particle can be determined from

$$m = \frac{p}{c}\sqrt{\frac{c^2 \mathrm{ToF}^2}{L^2}-1}$$

provided the path length and the momentum can be measured, say, by a tracking system.

What are typical ToF differences say for (charged) kaons and pions?
It is practical to perform the calculation as

$$\Delta \mathrm{ToF} = \frac{L}{c}(\frac{1}{\beta_1} - \frac{1}{\beta_2})\,,$$

with $\frac{1}{\beta} = \sqrt{1+m^2c^2/p^2}$.

In [ ]:
from hepunits import c_light, GeV, meter, ps, ns

import numpy as np

def ToF(m, p, L):
    """Time-of-Flight = particle path length L / (c * beta)"""
    # No c factors here because physicists give m and p without them, hence the c's cancel out, effectively ;-).
    one_over_beta = np.sqrt(1 + m*m/(p*p))
    return (L * one_over_beta /c_light)

For convenience, get hold of data information for the proton, K+ and pi+ (see `Particle`package down this notebook):

In [ ]:
from particle.particle.literals import proton, pi_plus, K_plus  # particle name literals

Calculate the difference in ToF between 10 GeV kaons and pions travelling over 10 meters:

In [ ]:
delta = ( ToF(K_plus.mass, 10*GeV, 10*meter) - ToF(pi_plus.mass, 10*GeV, 10*meter) ) / ps
print("At 10 GeV, Delta-TOF(K-pi) over 10 meters = {:.5} ps".format(delta))

Let's get a bit fancier:
- Compare protons, kaons and pions for a 1-meter path length.
- Look at the ToF difference versus momentum.
Other plotting tools (from HEP, actually) will be presented later on. For now let's just use the standard `matplotlib` library.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

p = np.arange(0.5, 5.1, 0.05) * GeV

# Calculate all the delta-ToF in picoseconds
delta1 = ( ToF(K_plus.mass, p, 1.*meter) - ToF(pi_plus.mass, p, 1.*meter) ) / ps
delta2 = ( ToF(proton.mass, p, 1.*meter) - ToF(K_plus.mass, p, 1.*meter) ) / ps
delta3 = ( ToF(proton.mass, p, 1.*meter) - ToF(pi_plus.mass, p, 1.*meter) ) / ps

fig, ax = plt.subplots()
ax.plot(p/GeV, delta1, label='K-$\pi$')
ax.plot(p/GeV, delta2, label='p-K')
ax.plot(p/GeV, delta3, label='p-$\pi$')

ax.set(xlabel='p [GeV]', ylabel='$\Delta$ ToF [ps]',
       title='Time-of-flight difference for a 1-meter path')
ax.grid()

plt.legend()
plt.ylim(bottom=0, top=500)
plt.show()

&nbsp;
<center><img src="images/logo_Particle.png" alt="Particle package logo" style="width: 150px;"/></center>

<center><h2><b><span style="color:green">PDG particle data, MC identification codes</span></b></h2></center>

### **Pythonic interface to**
- Particle Data Group (PDG) particle data table.
- Particle MC identification codes, with inter-MC converters.
- With various extra goodies.

### Package motivation - particle data

- The [PDG](http://pdg.lbl.gov/) provides a <span style="color:green">downloadable table of particle masses, widths, charges and Monte Carlo particle ID numbers</span> (PDG IDs).
  - Most recent file [here](http://pdg.lbl.gov/2020/html/computer_read.html).
- It <span style="color:green">also provided an experimental file with extended information</span>
(spin, quark content, P and C parities, etc.) until 2008 only, see [here](http://pdg.lbl.gov/2008/html/computer_read.html) (not widely known!).

- But <span style="color:green"><i>anyone</i> wanting to use these data</span>, the only readily available,
<span style="color:green">has to parse the file programmatically</span>.
- Why not make a Python package to deal with all these data, for everyone?

### Package motivation - MC identification codes

- The <span style="color:green">C++ HepPID and HepPDT libraries provide functions for processing particle ID codes</apan>
in the standard particle (aka PDG) numbering scheme.
- Different event generators may have their separate set of particle IDs: Geant3, etc.
- Again, why not make a package providing all functionality/conversions, Python-ically, for everyone?

### Package, in short

- <span style="color:green">Particle</span> - loads extended <b>PDG data tables</b> and implements search and manipulations / display.
- <span style="color:green">PDGID</span> - find out as much as possible from the PDG ID number. <b>No table lookup</b>.
- <span style="color:green">Converters for MC IDs</span> used in Pythia and Geant.
- Basic usage via the command line.
- Fexible / advanced usage programmatically.

### **1. `PDGID` class and MC ID classes**


- Classes `PDGID`, `PythiaID`, `Geant3ID`.
- Converters in module `particle.converters`: `Geant2PDGIDBiMap`, etc.

#### PDG IDs module overview

- <span style="color:green">Process and query PDG IDs</span>, and more – no look-up table needed.
  - Current version of package reflects the latest version of the
    <span style="color:green">HepPID & HepPDT utility functions</span> defined in the C++ HepPID and HepPDT versions 3.04.01
  - It contains more functionality than that available in the C++ code … and minor fixes too.
- Definition of a <span style="color:green">PDGID class, PDG ID literals</span>,
and set of standalone HepPID <span style="color:green">functions to query PDG IDs</span>
(is_meson, has_bottom, j_spin, charge, etc.).
   - All PDGID class functions are available standalone.

#### PDGID class
- Wrapper class `PDGID` for PDG IDs.
- Behaves like an int, with extra goodies.
- Large spectrum of properties and methods, with a Pythonic interface, and yet more!

In [ ]:
from particle import PDGID

In [ ]:
pid = PDGID(211)
pid

In [ ]:
PDGID(99999999)

In [ ]:
from particle.pdgid import is_meson

pid.is_meson, is_meson(pid)

To print all `PDGID` properties:

In [ ]:
print(pid.info())

#### MC ID classes and converters

- <span style="color:green">Classes for MC IDs</span> used in Pythia and Geant3: `PythiaID` and `Geant3ID`.
- <span style="color:green">ID converters</span> in module `particle.converters`: `Geant2PDGIDBiMap`, etc.

In [ ]:
from particle import PythiaID, Geant3ID

pyid = PythiaID(10221)

pyid.to_pdgid()

Conversions are directly available via mapping classes.

E.g., bi-directional map Pythia ID - PDG ID:

In [ ]:
from particle.converters import Pythia2PDGIDBiMap

Pythia2PDGIDBiMap[PDGID(9010221)]

In [ ]:
Pythia2PDGIDBiMap[PythiaID(10221)]

### **2. `Particle` class**

There are various ways to create a particle. The often used method is via its PDG ID.

In [ ]:
from particle import Particle

In [ ]:
Particle.from_pdgid(211)

**Searching**

<span style="color:green">Simple and natural API</span> to deal with the PDG particle data table,<br>with <span style="color:green">powerful 1-line search and look-up utilities!</span>

- `Particle.find(…)` – search a single match (exception raised if multiple particles match the search specifications).
- `Particle.findall(…)` – search a list of candidates.

- Search methods that can query any particle property!

In [ ]:
Particle.find('J/psi')

You can specify search terms as keywords - _any particle property_:

In [ ]:
Particle.find(latex_name=r'\phi(1020)')

You can directly check the numeric charge:

In [ ]:
Particle.findall('pi', charge=-1)

Or use a **lambda function** for the ultimate in generality! For example, to find all the neutral particles with a bottom quark between 5.2 and 5.3 GeV:

In [ ]:
from hepunits import GeV, s  # Units are good. Use them.

In [ ]:
Particle.findall(lambda p:
                     p.pdgid.has_bottom
                     and p.charge==0
                     and 5.2*GeV < p.mass < 5.3*GeV
                )

Another lambda function example: You can use the width or the lifetime:

In [ ]:
Particle.findall(lambda p: p.lifetime > 1000*s)

If you want infinite lifetime, you could just use the keyword search instead:

In [ ]:
Particle.findall(lifetime=float('inf'))

Trivially find all pseudoscalar charm mesons:

In [ ]:
from particle import SpinType

Particle.findall(lambda p: p.pdgid.is_meson and p.pdgid.has_charm and p.spin_type==SpinType.PseudoScalar)

**Display**

Nice display in Jupyter notebooks, as well as `str` and `repr` support:

In [ ]:
p = Particle.from_pdgid(-415)
p

In [ ]:
print(p)

In [ ]:
print(repr(p))

Full descriptions:

In [ ]:
print(p.describe())

You may find LaTeX or HTML to be more useful in your program; both are supported:

In [ ]:
print(p.latex_name, '\n', p.html_name)

**Particle properties**

You can do things to particles, like **invert** them:

In [ ]:
~p

There are a plethora of properties you can access:

In [ ]:
p.spin_type

You can quickly access the PDGID of a particle:

In [ ]:
p.pdgid

### **3. Literals**

They provide a <span style="color:green">handy way to manipulate things with human-readable names!</span>

`Particle` defines <span style="color:green">literals for most common particles</span>, with easily recognisable names.
- Literals are dynamically generated on import for both `PDGID` and `Particle` classes.

**PDGID literals**

In [ ]:
from particle.pdgid import literals as lid

In [ ]:
lid.phi_1020

**Particle literals**

In [ ]:
from particle import literals as lp

In [ ]:
lp.phi_1020

### **4. Data files, stored in `particle/data/`**

- <b>PDG particle data files</b>
  - Original PDG data files, which are in a fixed-width format - simply for bookkeeping and reference.
  - Code rather uses “digested forms” of these, produced within `Particle`, stored as CSV, for optimised querying.
  - Latest PDG data (2020) used by default.
  - Advanced usage: user can load older PDG tables, load a “user table” with new particles, append to default table.

- <b>Other data files</b>
  - CSV file for mapping of PDG IDs to particle LaTeX names.

**Dump table contents**

The package provides the 2 methods `Particle.to_dict(...)` and `Particle.to_list(...)`, which make it easy to dump (selected) particle properties in an easy way. No need to dig into the package installation directory to inspect the particle data table ;-).

Tabular output can be formatted with the powerful package `tabulate`, for example (other similar libraries exist).

In [ ]:
help(Particle.to_dict)

In [ ]:
from tabulate import tabulate

In [ ]:
fields = ['pdgid', 'pdg_name', 'mass', 'mass_upper', 'mass_lower', 'three_charge']
query_as_dict = Particle.to_dict(exclusive_fields=fields, n_rows=10)
print(tabulate(query_as_dict, headers='keys'))

Be fancy - table with all pseudoscalar charm hadrons, in _reStructuredText_ format:

In [ ]:
fields = ['pdgid', 'name', 'evtgen_name', 'mass', 'mass_upper', 'mass_lower', 'three_charge']

query_as_dict = Particle.to_dict(filter_fn=lambda p: p.pdgid.is_meson and p.pdgid.has_charm and p.spin_type==SpinType.PseudoScalar,
                                 exclusive_fields=fields)
print(tabulate(query_as_dict, headers='keys', tablefmt='rst'))

Notebook-friendly HTML is just as easy:

In [ ]:
from IPython.display import HTML

query_as_dict = Particle.to_dict(filter_fn=lambda p: p.pdgid.is_meson and p.pdgid.has_charm and p.spin_type==SpinType.PseudoScalar,
                                 exclusive_fields=['pdgid', 'pdg_name', 'html_name'])
HTML(tabulate(query_as_dict, headers='keys', tablefmt='html'))

### **5. Advanced usage**

You can:

* Extend or replace the default particle data table in `Particle`.
* Adjust properties for a particle.
* Make custom particles.

&nbsp;<br>
<center>
    <img src="images/logo_decaylanguage.png" alt="DecayLanguage package logo" style="width: 200px;"/>
    <h2><b><span style="color:green">Decay files, universal description of decay chains</span></b></h2>
</center>

`DecayLanguage` is designed for the manipulation of decay structures in Python. The current package has:

- Decay file parsers:
  - Read *.dec DecFiles*, such as EvtGen decay files typically used in Flavour Physics experiments.
  - Manipulate and visualise them in Python.
- Amplitude Analysis decay language:
  - Input based on AmpGen generator, output format for GooFit C++ program.

### Package motivation

- Ability to describe decay-tree-like structures.
- Provide a translation of decay amplitude models from AmpGen to GooFit.
  - Idea is to generalise this to other decay descriptions.
- Any experiment uses event generators which, among many things, need to describe particle decay chains.
- Programs such as EvtGen rely on so-called .dec decay files.
- Many experiments need decay data files.
- Why not make a Python package to deal with decay files, for everyone?

### Package, in short

- Tools to parse decay files and programmatically manipulate them, query, display information.
  - Descriptions and parsing built atop the [Lark parser](https://github.com/lark-parser/lark/).
- Tools to translate decay amplitude models from AmpGen to GooFit, and manipulate them.

### **1. Decay files**

#### *Master file” DECAY.DEC

<span style="color:green">Gigantic file defining decay modes for all relevant particles, including decay model specifications.</span>
LHCb uses one. Belle II as well, and others.

#### User .dec files
- Needed to produce specific MC samples.
- Typically contain a single decay chain (except if defining inclusive samples).

**Example user decay file:**

<small>
<pre>
# Decay file for [B_c+ -> (B_s0 -> K+ K-) pi+]cc

Alias      B_c+sig        B_c+
Alias      B_c-sig        B_c-
ChargeConj B_c+sig        B_c-sig
Alias      MyB_s0         B_s0
Alias      Myanti-B_s0    anti-B_s0
ChargeConj MyB_s0         Myanti-B_s0

Decay B_c+sig
  1.000     MyB_s0     pi+     PHOTOS PHSP;
Enddecay
CDecay B_c-sig

Decay MyB_s0
    1.000     K+     K-     SSD_CP 20.e12 0.1 1.0 0.04 9.6 -0.8 8.4 -0.6;
Enddecay
CDecay Myanti-B_s0
</pre>
</small>

### **2. Decay file parsing**

- **Parsing should be simple**
  - Expert users can configure parser choice and settings, etc. 
- **Parsing should be (reasonably) fast!**

After parsing, many queries are possible!

In [ ]:
from decaylanguage import DecFileParser

#### The LHCb "master decay file"

It's a big file! ~ 500 particle decays defined, thousands of decay modes, over 11k lines in total.

In [ ]:
dfp = DecFileParser('data/DECAY_LHCB.DEC')

In [ ]:
%%time
dfp.parse()

In [ ]:
dfp

Let's parse and play with a small decay file:

In [ ]:
with open('data/Dst.dec') as f:
    print(f.read())

In [ ]:
dfp_Dst = DecFileParser('data/Dst.dec')
dfp_Dst

In [ ]:
dfp_Dst.parse()
dfp_Dst

It can be handy to **parse from a multi-line string** rather than a file:

In [ ]:
s = """
# Decay file for [B_c+ -> (B_s0 -> K+ K-) pi+]cc

Alias      B_c+sig        B_c+
Alias      B_c-sig        B_c-
ChargeConj B_c+sig        B_c-sig
Alias      MyB_s0         B_s0
Alias      Myanti-B_s0    anti-B_s0
ChargeConj MyB_s0         Myanti-B_s0

Decay B_c+sig
  1.000     MyB_s0     pi+     PHOTOS PHSP;
Enddecay
CDecay B_c-sig

Decay MyB_s0
    1.000     K+     K-     SSD_CP 20.e12 0.1 1.0 0.04 9.6 -0.8 8.4 -0.6;
Enddecay
CDecay Myanti-B_s0
"""

In [ ]:
dfp = DecFileParser.from_string(s)
dfp.parse()
dfp

#### Decay file information

In [ ]:
dfp_Dst.print_decay_modes('D*+')

In [ ]:
dfp_Dst.list_decay_mother_names()

In [ ]:
dfp_Dst.list_decay_modes('D*+')

#### Info such as particle aliases

In [ ]:
dfp.dict_aliases()

In [ ]:
dfp.dict_charge_conjugates()

### **3.  Display of decay chains**

The parser can provide a simple `dict` representation of any decay chain found in the input decay file(s). Being generic and simple, that is what is used as input information for the viewer class (see below).

In [ ]:
dc = dfp_Dst.build_decay_chains('D+')
dc

In [ ]:
from decaylanguage import DecayChainViewer

In [ ]:
DecayChainViewer(dc)

In [ ]:
DecayChainViewer(dfp_Dst.build_decay_chains('D*+'))

In [ ]:
dc = dfp_Dst.build_decay_chains('D*+', stable_particles=['D+', 'D0', 'pi0'])
DecayChainViewer(dc)

#### **Charge conjugation**

In [ ]:
dc_cc = dfp_Dst.build_decay_chains('D*-', stable_particles=['D-', 'anti-D0', 'pi0'])
DecayChainViewer(dc_cc)

#### **Parsing several files**

Typically useful when the user decay file needs information from the master decay file.

In [ ]:
s = u"""
Alias      MyXic+              Xi_c+
Alias      MyantiXic-          anti-Xi_c-
ChargeConj MyXic+              MyantiXic-

Decay Xi_cc+sig
  1.000       MyXic+    pi-    pi+       PHSP;
Enddecay
CDecay anti-Xi_cc-sig

Decay MyXic+
  1.000       p+    K-    pi+       PHSP;
Enddecay
CDecay MyantiXic-

End
"""

In [ ]:
dfp = DecFileParser.from_string(s)
dfp.parse()
dfp

Note the subtletly: 3, not 4 decays, are found! This is because the file contains no statement
`ChargeConj anti-Xi_cc-sigXi_cc+sig`, hence the parser cannot know to which particle (matching `Decay` statement) the charge-conjugate decay of `anti-Xi_cc-sig` relates to (code does not rely on position of statements to guess ;-)).

In [ ]:
d = dfp.build_decay_chains('Xi_cc+sig')
DecayChainViewer(d)

As said in the warning, the information provided is not enough for the anti-Xi_cc-sig to make sense:

In [ ]:
from decaylanguage.dec.dec import DecayNotFound

try:
    d = dfp.build_decay_chains('anti-Xi_cc-sig')
except DecayNotFound:
    print("Decays of particle 'anti-Xi_cc-sig' not found in .dec file!")

But the missing information is easily providing **parsing two files simultaneously ...!** (Any number of files is allowed.)

In [ ]:
from tempfile import NamedTemporaryFile

with NamedTemporaryFile(delete=False) as tf:
    tf.write(s.encode('utf-8'))
    
dfp = DecFileParser(tf.name, 'data/DECAY_LHCB.DEC')
dfp.parse()

In [ ]:
dc = dfp.build_decay_chains('Xi_cc+sig')

DecayChainViewer(dc)

In [ ]:
dc_cc = dfp.build_decay_chains('anti-Xi_cc-sig')

DecayChainViewer(dc_cc)

Want to save a graph? Try for example 
```
dcv = DecayChainViewer(...)
dcv.graph.write_pdf('test.pdf')
```

### **4. Representation of decay chains**

<span style="color:green">The universal (and digital) representation of decay chains is of interest well outside the context of decay file parsing!</span>

#### Building blocks

- A <span style="color:green">daughters list</span> - list of final-state particles.
- A <span style="color:green">decay mode</span> - typically a branching fraction and a list of final-state particles (may also contain _any_ metadata such as decay model and optional decay-model parameters, as defined for example in .dec decay files).
- A <span style="color:green">decay chain</span> - can be seen as a mother particle and a list of decay modes.

In [ ]:
from decaylanguage.decay.decay import DaughtersDict, DecayMode, DecayChain

**Daughters list** (actually a ``Counter`` dictionary, internally):

In [ ]:
# Constructor from a dictionary
dd = DaughtersDict({'K+': 1, 'K-': 2, 'pi+': 1, 'pi0': 1})

# Constructor from a list of particle names
dd = DaughtersDict(['K+', 'K-', 'K-', 'pi+', 'pi0'])

# Constructor from a string representing the final state
dd = DaughtersDict('K+ K- pi0')
dd

#### Decay Modes

In [ ]:
# A 'default' and hence empty, decay mode
dm = DecayMode()

# Decay mode with minimal input information
dd = DaughtersDict('K+ K-')
dm = DecayMode(0.5, dd)

# Decay mode with decay model information and user metadata
dm = DecayMode(0.2551,                                              # branching fraction
               'pi- pi0 nu_tau',                                    # final-state particles
               model='TAUHADNU',                                    # decay model
               model_params=[-0.108, 0.775, 0.149, 1.364, 0.400],   # decay-model parameters
               study='toy', year=2019                               # user metadata
              )
dm

In [ ]:
print(dm.describe())

Various manipulations are available:

In [ ]:
dm = DecayMode.from_pdgids(0.5, [321, -321])
print(dm)

dm = DecayMode(1.0, 'K+ K+ pi-')
dm.charge_conjugate()

#### Decay chains 

In [ ]:
dm1 = DecayMode(0.0124, 'K_S0 pi0', model='PHSP')
dm2 = DecayMode(0.692, 'pi+ pi-')
dm3 = DecayMode(0.98823, 'gamma gamma')
dc = DecayChain('D0', {'D0':dm1, 'K_S0':dm2, 'pi0':dm3})

dc

In [ ]:
dc.decays

Flatten the decay chain, i.e. replace all intermediate, decaying particles, with their final states:
- The BF is now the *visible BF*

In [ ]:
dc.flatten()

Of course you can sill just as easily visualise decays defined via this `DecayChain` class:

In [ ]:
DecayChainViewer(dc.to_dict())